# CoSim - RTED

In [1]:
import numpy as np

import andes
import ams

import pandas as pd

import json

import cvxpy as cp

In [2]:
ams.__version__

'0.6.5.post85.dev0+gfd25f07'

In [3]:
andes.config_logger(stream_level=20)

In [4]:
ams.config_logger(stream_level=10)

In [5]:
sp = ams.load(ams.get_case('ieee14/ieee14_rted.xlsx'),
              setup=True,
              )

Working directory: "/home/jwang/work/ams/dev/demo"
Input format guessed as xlsx.
Parsing input file "/home/jwang/work/ams/ams/cases/ieee14/ieee14_rted.xlsx"...
Input file parsed in 0.2552 seconds.
Adjust bus index to start from 0.
System set up in 0.0152 seconds.


In [6]:
sp.RTED.syms.generate_symbols()

- Generating symbols for RTED


In [7]:
sp.RTED.run()

Setup model of RTED
RTED data check passed.
- Generating symbols for RTED
Set constrs pb: sum(pd1) + sum(pd2) - sum(pg)
Set constrs lub: PTDF1 @ (pg - pd1) - PTDF2 * pd2 - rate_a
Set constrs llb: - PTDF1 @ (pg - pd1) + PTDF2 * pd2 - rate_a
Set constrs rbu: gs @ pru - du
Set constrs rbd: gs @ prd - dd
Set constrs rru: pg + pru - pmax
Set constrs rrd: -pg + prd - pmin
Set constrs rgu: pg - pg0 - R10
Set constrs rgd: -pg + pg0 - R10
RTED model set up in 0.0274 seconds.
RTED solved as optimal in 0.0276 seconds with exit code 0.


True

In [8]:
sp.RTED.dc2ac()

Setup model for ACOPF
ACOPF data check passed.
- Generating symbols for ACOPF
Set constrs pb: sum( pd1 ) +sum( pd2 ) - sum( pg )
ACOPF model set up in 0.0084 seconds.
Adjust bus index to start from 0.
PYPOWER Version 5.1.4, 27-June-2018
 -- AC Optimal Power Flow



Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011


ACOPF completed in 0.7179 seconds with exit code 0.
RTED is converted to AC.


Converged!


True

In [9]:
sa = sp.to_andes(setup=False,
                 addfile=andes.get_case('ieee14/ieee14_wt3.xlsx'),
                 overwrite=True,
                 keep=False,
                 no_output=True,
                 )

sa.add("Toggle", dict(model='SynGen', dev="GENROU_3", t=11.0))

sa.setup()

sp.dyn.link_andes(sa)

JSON file written to "ieee14_rted.json"
System convert to ANDES in 0.1589 seconds, saved as "ieee14_rted.json".
Working directory: "/home/jwang/work/ams/dev/demo"
> Loaded config from file "/home/jwang/.andes/andes.rc"
> Loaded generated Python code in "/home/jwang/.andes/pycode".
Parsing input file "ieee14_rted.json"...
Input file parsed in 0.0032 seconds.
Converted file is removed. Set "no_keep=False" to keep it.
Addfile format guessed as xlsx.
Parsing additional file "/home/jwang/.local/lib/python3.10/site-packages/andes/cases/ieee14/ieee14_wt3.xlsx"...
Power flow models exist in the addfile. Only dynamic models will be used.
Adjust IdxParam <GENROU.gen>
Adjust IdxParam <REGCA1.gen>
Addfile parsed in 0.1852 seconds.
System has not been linked to ANDES. Call ``dyn.link_andes(adsys=sa)`` after setup ANDES system.
System internal structure set up in 0.0571 seconds.
AMS system 0x7f9a402cc7f0 is linked to the ANDES system 0x7f9955fc51e0.


In [10]:
sp.dyn.send()

RTED results has been converted to AC.
Sending RTED results to ANDES <0x7f9955fc51e0>...
Sending results to <pflow> models...
Var <RTED.vBus> is mapped from <Bus.v0>.


True

In [11]:
sa.PFlow.run()

-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: Off
   Sparse solver: KLU
 Solution method: NR method
Power flow initialized in 0.0089 seconds.
0: |F(x)| = 0.5605182162
1: |F(x)| = 0.006202200463
2: |F(x)| = 5.819382991e-06
3: |F(x)| = 6.960640397e-12
Converged in 4 iterations in 0.0133 seconds.


True

In [12]:
idx_tg = sa.TGOV1.idx.v
vmax0 = sa.TGOV1.get(src='VMAX', attr='v', idx=idx_tg)
sa.TGOV1.set(src='VMAX', attr='v', idx=idx_tg, value=10 * vmax0)
vmin0 = sa.TGOV1.get(src='VMIN', attr='v', idx=idx_tg)
sa.TGOV1.set(src='VMIN', attr='v', idx=idx_tg, value=np.zeros_like(vmin0))

True

In [13]:
sa.TDS.config.criteria = 0

In [14]:
sa.TDS.config.tf = 10
sa.TDS.run()


-> Time Domain Simulation Summary:
Sparse Solver: KLU
Simulation time: 0.0-10 s.
Fixed step size: h=33.33 ms. Shrink if not converged.
Initialization for dynamics completed in 0.1329 seconds.
Initialization was successful.


  0%|          | 0/100 [00:00<?, ?%/s]

<Toggle Toggler_1>: Line.Line_1 status changed to 0 at t=1.0 sec.
<Toggle Toggler_2>: Line.Line_1 status changed to 1 at t=1.1 sec.


Simulation to t=10.00 sec completed in 2.1539 seconds.


True

In [15]:
sa.TDS.config.tf = 15
sa.TDS.run()

  0%|          | 0/100 [00:00<?, ?%/s]

<Toggle Toggle_3>: SynGen.GENROU_3 status changed to 0 at t=11.0 sec.


Simulation to t=15.00 sec completed in 2.3667 seconds.


True

In [16]:
sa.GENROU.as_df()

,idx,u,name,bus,gen,coi,coi2,Sn,Vn,fn,...,gammaq,xd,xq,xd2,xq1,xq2,Td10,Td20,Tq10,Tq20
uid,,,,,,,,,,,,,,,,,,,,,
0,GENROU_1,1.0,GENROU_1,1,Slack_1,None,None,100.0,69.0,60.0,...,1.0,1.8,1.75,0.23,0.8,0.23,6.5,0.06,0.2,0.05
1,GENROU_2,1.0,GENROU_2,2,PV_2,None,None,100.0,69.0,60.0,...,1.0,1.8,1.75,0.28,0.8,0.28,6.5,0.06,0.2,0.05
2,GENROU_3,0.0,GENROU_3,3,PV_3,None,None,100.0,69.0,60.0,...,1.0,1.8,1.75,0.34,0.8,0.34,6.5,0.06,0.2,0.05
3,GENROU_4,1.0,GENROU_4,6,PV_4,None,None,100.0,138.0,60.0,...,1.0,1.8,1.75,0.28,0.8,0.28,6.5,0.06,0.2,0.05


In [17]:
sp.dyn.receive()

Receiving ANDES <0x7f9955fc51e0> results to RTED.
Receiving <tds> results to RTED...
Generator online status are received from dynamic generators.
Generator output power are received from dynamic generators.


True

In [18]:
sa.GENROU.u

NumParam: GENROU.u, v=[1. 1. 0. 1.], vin=[1. 1. 1. 1.]

In [19]:
sp.PV.u

NumParam: PV.u, v=[1. 0. 1. 1.], vin=[1. 1. 1. 1.]

In [20]:
sp.RTED.vars

OrderedDict([('pg',
              Var: StaticGen.pg, a=[0 1 2 3 4], v=[0.49997832 0.11666717 0.1500005  0.10000051 1.42711649]),
             ('pru',
              Var: StaticGen.pru, a=[5 6 7 8 9], v=[0.   0.   0.   0.05 0.05]),
             ('prd',
              Var: StaticGen.prd, a=[10 11 12 13 14], v=[0.01666667 0.01666667 0.05       0.         0.01666667]),
             ('vBus',
              Var: Bus.vBus, a=[14 15 16 17 18 19 20 21 22 23 24 25 26 27], v=[1.09999741 1.08292312 1.05407319 1.05693267 1.06400603 1.06136747
               1.07097538 1.08353591 1.06768589 1.05919061 1.05685487 1.05050197
               1.04884531 1.05926167])])

In [21]:
sp.RTED.run()

RTED data check passed.
- Generating symbols for RTED
Set constrs pb: sum(pd1) + sum(pd2) - sum(pg)
Set constrs lub: PTDF1 @ (pg - pd1) - PTDF2 * pd2 - rate_a
Set constrs llb: - PTDF1 @ (pg - pd1) + PTDF2 * pd2 - rate_a
Set constrs rbu: gs @ pru - du
Set constrs rbd: gs @ prd - dd
Set constrs rru: pg + pru - pmax
Set constrs rrd: -pg + prd - pmin
Set constrs rgu: pg - pg0 - R10
Set constrs rgd: -pg + pg0 - R10
RTED model set up in 0.0297 seconds.
RTED failed as infeasible with exit code 1!


False

In [22]:
sp.RTED.pg

Var: StaticGen.pg, a=[0 1 2 3 4], v=None

In [23]:
sp.dyn.send()

RTED is not solved optimally.


False